In [7]:
import spotty
import tensorflow as tf
import subprocess
import tempfile
import os
import json
import time
import importlib

In [8]:
importlib.reload(spotty)

<module 'spotty' from '/home/ubuntu/landmarks/spotty.py'>

In [9]:
def get_price():
    response = subprocess.run(['aws', 'ec2', 'describe-spot-price-history', '--instance-types', 'p3.2xlarge'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    history = json.loads(response.stdout.decode('utf-8'))
    latest_price = list(filter(lambda x: x['AvailabilityZone'] == 'us-west-2a', history['SpotPriceHistory']))[0]['SpotPrice']
    latest_price = float(latest_price)
    return str(round(latest_price * 1.05, 2))

bid_price = get_price()
bid_price

'1.21'

In [4]:
# ami = "ami-c91677b1" most recent Amazon AMI
ami = "ami-880e64f0" # above AMI w/ some extra python packages
# instance_type="p2.xlarge"
instance_type="p3.2xlarge"
instance_spec = {
    'ImageId': ami,
    'KeyName': 'deep-oregon',
    'SecurityGroupIds': ['sg-54b0802b'],
    'InstanceType': instance_type,
    'SubnetId':'subnet-29845850',
    'Placement': {
        'AvailabilityZone': 'us-west-2a'
    }
}
with tempfile.NamedTemporaryFile() as f:
    f.write(json.dumps(instance_spec).encode('utf-8'))
    f.flush()
    response = subprocess.run(['aws', 'ec2', 'request-spot-instances', '--instance-count', '1', 
                               '--spot-price', bid_price, '--launch-specification', 
                               'file://' + os.path.join(tempfile.gettempdir(), f.name), '--type', 'one-time'], 
                              stdout=subprocess.PIPE, stderr=subprocess.PIPE)

error = response.stderr.decode('utf-8')
if error: print('encountered error', error)
result = response.stdout.decode('utf-8')
result = json.loads(result)
request_id = result['SpotInstanceRequests'][0]['SpotInstanceRequestId']
instance_id = spotty.find_instance_id(request_id)
instance_id

'i-01140d8427ff03766'

In [5]:
devices = [{'aws':'/dev/sdf', 'ubuntu': '/dev/xvdf', 'dir': '/data', 'volume_id': 'vol-023fe5bce0bc42fe3'},
{'aws':'/dev/sdg', 'ubuntu': '/dev/xvdg', 'dir': '/home/ubuntu/notebook', 'volume_id':'vol-095fa9b501c909d92'}]
for device in devices:
    result = subprocess.run(['aws', 'ec2', 'attach-volume', '--volume-id', device['volume_id'], '--instance-id', instance_id, "--device", device['aws']], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(result.stdout)
    print(result.stderr)
    #TODO use aws ec2 describe-volumes instead of error prone sleep
    time.sleep(20)
    spotty.run_command(instance_id, ['sudo', 'mkdir', '-p', device['dir']])
    spotty.run_command(instance_id, ['sudo', 'mount', device['ubuntu'], device['dir']])

b'{\n    "AttachTime": "2018-06-01T18:21:12.307Z",\n    "Device": "/dev/sdf",\n    "InstanceId": "i-01140d8427ff03766",\n    "State": "attaching",\n    "VolumeId": "vol-023fe5bce0bc42fe3"\n}\n'
b''
running commmand ['sudo', 'mkdir', '-p', '/data']
STDOUT b''
STDERR b"Warning: Permanently added '52.42.10.2' (ECDSA) to the list of known hosts.\r\nsudo: unable to resolve host ip-172-30-0-250\n"
running commmand ['sudo', 'mount', '/dev/xvdf', '/data']
STDOUT b''
STDERR b'sudo: unable to resolve host ip-172-30-0-250\n'
b'{\n    "AttachTime": "2018-06-01T18:21:35.794Z",\n    "Device": "/dev/sdg",\n    "InstanceId": "i-01140d8427ff03766",\n    "State": "attaching",\n    "VolumeId": "vol-095fa9b501c909d92"\n}\n'
b''
running commmand ['sudo', 'mkdir', '-p', '/home/ubuntu/notebook']
STDOUT b''
STDERR b'sudo: unable to resolve host ip-172-30-0-250\n'
running commmand ['sudo', 'mount', '/dev/xvdg', '/home/ubuntu/notebook']
STDOUT b''
STDERR b'sudo: unable to resolve host ip-172-30-0-250\n'


In [6]:
spotty.describe_instance(instance_id)

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
       'Ebs': {'AttachTime': '2018-06-01T18:20:07.000Z',
        'DeleteOnTermination': True,
        'Status': 'attached',
        'VolumeId': 'vol-0434b10f29ae3f8aa'}},
      {'DeviceName': '/dev/sdf',
       'Ebs': {'AttachTime': '2018-06-01T18:21:12.000Z',
        'DeleteOnTermination': False,
        'Status': 'attached',
        'VolumeId': 'vol-023fe5bce0bc42fe3'}},
      {'DeviceName': '/dev/sdg',
       'Ebs': {'AttachTime': '2018-06-01T18:21:35.000Z',
        'DeleteOnTermination': False,
        'Status': 'attached',
        'VolumeId': 'vol-095fa9b501c909d92'}}],
     'ClientToken': 'ea1edd8c67feead93fd3f2240a17eb9539bb4a3445f15118b81a0ca11bf3',
     'EbsOptimized': False,
     'EnaSupport': True,
     'Hypervisor': 'xen',
     'ImageId': 'ami-880e64f0',
     'InstanceId': 'i-01140d8427ff03766',
     'InstanceLifecyc